In [1]:
import numpy as np
import pandas as pd

#最大表示列数の指定（ここでは50列を指定）
pd.set_option('display.max_columns', 100)

import pprint
from collections import OrderedDict

In [2]:
# トレーニングデータ、テストデータ、サンプルサブミットデータを読み込み
train = pd.read_csv('train_data.csv')
test = pd.read_csv('test_data.csv')

In [3]:
print(train.shape) # 学習用データ
print(test.shape) # 本番予測用データ

(66125, 32)
(28340, 31)


In [4]:
print(train.columns) # トレーニングデータの列名
print('-' * 10) # 区切り線を表示
print(test.columns) # テストデータの列名

Index(['id', 'period', 'game-ver', 'lobby-mode', 'lobby', 'mode', 'stage',
       'A1-weapon', 'A1-rank', 'A1-level', 'A2-weapon', 'A2-rank', 'A2-level',
       'A3-weapon', 'A3-rank', 'A3-level', 'A4-weapon', 'A4-rank', 'A4-level',
       'B1-weapon', 'B1-rank', 'B1-level', 'B2-weapon', 'B2-rank', 'B2-level',
       'B3-weapon', 'B3-rank', 'B3-level', 'B4-weapon', 'B4-rank', 'B4-level',
       'y'],
      dtype='object')
----------
Index(['id', 'period', 'game-ver', 'lobby-mode', 'lobby', 'mode', 'stage',
       'A1-weapon', 'A1-rank', 'A1-level', 'A2-weapon', 'A2-rank', 'A2-level',
       'A3-weapon', 'A3-rank', 'A3-level', 'A4-weapon', 'A4-rank', 'A4-level',
       'B1-weapon', 'B1-rank', 'B1-level', 'B2-weapon', 'B2-rank', 'B2-level',
       'B3-weapon', 'B3-rank', 'B3-level', 'B4-weapon', 'B4-rank', 'B4-level'],
      dtype='object')


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66125 entries, 0 to 66124
Data columns (total 32 columns):
id            66125 non-null int64
period        66125 non-null object
game-ver      66125 non-null object
lobby-mode    66125 non-null object
lobby         66125 non-null object
mode          66125 non-null object
stage         66125 non-null object
A1-weapon     66125 non-null object
A1-rank       51681 non-null object
A1-level      66125 non-null int64
A2-weapon     66125 non-null object
A2-rank       51681 non-null object
A2-level      66125 non-null float64
A3-weapon     66125 non-null object
A3-rank       51681 non-null object
A3-level      66125 non-null float64
A4-weapon     66074 non-null object
A4-rank       51638 non-null object
A4-level      66074 non-null float64
B1-weapon     66125 non-null object
B1-rank       51681 non-null object
B1-level      66125 non-null int64
B2-weapon     66125 non-null object
B2-rank       51681 non-null object
B2-level      66125 non-nul

In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28340 entries, 0 to 28339
Data columns (total 31 columns):
id            28340 non-null int64
period        28340 non-null object
game-ver      28340 non-null object
lobby-mode    28340 non-null object
lobby         28340 non-null object
mode          28340 non-null object
stage         28340 non-null object
A1-weapon     28340 non-null object
A1-rank       22068 non-null object
A1-level      28340 non-null int64
A2-weapon     28340 non-null object
A2-rank       22068 non-null object
A2-level      28340 non-null float64
A3-weapon     28340 non-null object
A3-rank       22068 non-null object
A3-level      28340 non-null float64
A4-weapon     28325 non-null object
A4-rank       22060 non-null object
A4-level      28325 non-null float64
B1-weapon     28340 non-null object
B1-rank       22068 non-null object
B1-level      28340 non-null int64
B2-weapon     28340 non-null object
B2-rank       22068 non-null object
B2-level      28340 non-nul

In [7]:
train.head()

,id,period,game-ver,lobby-mode,lobby,mode,stage,A1-weapon,A1-rank,A1-level,A2-weapon,A2-rank,A2-level,A3-weapon,A3-rank,A3-level,A4-weapon,A4-rank,A4-level,B1-weapon,B1-rank,B1-level,B2-weapon,B2-rank,B2-level,B3-weapon,B3-rank,B3-level,B4-weapon,B4-rank,B4-level,y
0,1,2019-10-15T20:00:00+00:00,5.0.1,regular,standard,nawabari,sumeshi,sshooter_becchu,NaN,139,soytuber_custom,NaN,118.0,pablo_hue,NaN,13.0,hokusai,NaN,10.0,bold_7,NaN,28,hokusai_becchu,NaN,26.0,herocharger_replica,NaN,68.0,sharp_neo,NaN,31.0,1
1,2,2019-12-14T04:00:00+00:00,5.0.1,regular,standard,nawabari,arowana,parashelter_sorella,NaN,198,jetsweeper,NaN,77.0,campingshelter_camo,NaN,198.0,nzap85,NaN,123.0,momiji,NaN,83,squiclean_b,NaN,118.0,campingshelter,NaN,168.0,sputtery_clear,NaN,151.0,0
2,3,2019-12-25T14:00:00+00:00,5.0.1,gachi,standard,hoko,ama,nzap89,a-,114,quadhopper_black,a,68.0,prime_becchu,a,225.0,jetsweeper,a,107.0,bold_7,a-,50,nzap85,a+,163.0,prime_becchu,a-,160.0,dualsweeper_custom,a,126.0,0
3,4,2019-11-11T14:00:00+00:00,5.0.1,regular,standard,nawabari,engawa,bamboo14mk1,NaN,336,splatroller_becchu,NaN,131.0,dynamo_tesla,NaN,189.0,prime_becchu,NaN,41.0,splatcharger_becchu,NaN,273,liter4k,NaN,189.0,promodeler_mg,NaN,194.0,hotblaster_custom,NaN,391.0,0
4,5,2019-12-14T06:00:00+00:00,5.0.1,gachi,standard,hoko,chozame,bold_7,x,299,hissen_hue,x,97.0,h3reelgun_d,x,96.0,splatroller,x,136.0,furo,x,101,sputtery_hue,x,45.0,bucketslosher_soda,x,246.0,wakaba,x,160.0,1


In [8]:
test.head()

,id,period,game-ver,lobby-mode,lobby,mode,stage,A1-weapon,A1-rank,A1-level,A2-weapon,A2-rank,A2-level,A3-weapon,A3-rank,A3-level,A4-weapon,A4-rank,A4-level,B1-weapon,B1-rank,B1-level,B2-weapon,B2-rank,B2-level,B3-weapon,B3-rank,B3-level,B4-weapon,B4-rank,B4-level
0,1,2019-12-17T12:00:00+00:00,5.0.1,gachi,standard,area,hakofugu,prime_collabo,x,174,herospinner_replica,x,130.0,nzap89,x,127.0,nova_becchu,x,233.0,furo,x,160,nautilus47,x,151.0,l3reelgun_d,x,213.0,nzap89,x,306.0
1,2,2019-11-25T16:00:00+00:00,5.0.1,gachi,standard,asari,anchovy,prime_becchu,s+,363,nzap89,s+,59.0,dynamo_becchu,s+,36.0,rapid_becchu,s+,225.0,furo,s+,326,dualsweeper_custom,s+,289.0,prime_becchu,s+,147.0,splatroller,s+,156.0
2,3,2019-10-22T08:00:00+00:00,5.0.1,gachi,standard,area,mutsugoro,furo_deco,s,116,bold,s,125.0,nzap85,s,124.0,wakaba,s,181.0,splatroller,s,76,momiji,s,232.0,nzap83,s,183.0,promodeler_pg,s,105.0
3,4,2019-12-30T04:00:00+00:00,5.0.1,gachi,standard,asari,devon,prime_becchu,s+,192,splatspinner_collabo,s+,115.0,pablo,s+,171.0,dualsweeper_custom,s+,170.0,momiji,s+,90,quadhopper_black,s+,119.0,soytuber,s+,172.0,prime,s+,70.0
4,5,2019-10-15T16:00:00+00:00,5.0.1,gachi,standard,yagura,anchovy,l3reelgun_d,x,267,rapid_becchu,x,223.0,heroroller_replica,x,240.0,hydra_custom,x,190.0,nzap83,x,259,sshooter_becchu,x,198.0,splatscope,x,50.0,screwslosher_becchu,x,287.0


In [9]:
train.isnull().sum() 

id                0
period            0
game-ver          0
lobby-mode        0
lobby             0
mode              0
stage             0
A1-weapon         0
A1-rank       14444
A1-level          0
A2-weapon         0
A2-rank       14444
A2-level          0
A3-weapon         0
A3-rank       14444
A3-level          0
A4-weapon        51
A4-rank       14487
A4-level         51
B1-weapon         0
B1-rank       14444
B1-level          0
B2-weapon         0
B2-rank       14444
B2-level          0
B3-weapon         1
B3-rank       14444
B3-level          1
B4-weapon        67
B4-rank       14501
B4-level         67
y                 0
dtype: int64

In [10]:
# trainとtestを縦に連結
df_full = pd.concat([train, test], axis=0, sort=False)
print(df_full.shape) # df_fullの行数と列数を確認
df_full.describe() # df_fullの要約統計量

(94465, 32)


,id,A1-level,A2-level,A3-level,A4-level,B1-level,B2-level,B3-level,B4-level,y
count,94465.000000,94465.000000,94465.000000,94465.000000,94399.000000,94465.000000,94465.000000,94463.000000,94368.000000,66125.000000
mean,27395.150003,208.713979,155.143217,153.007230,155.815432,156.597502,152.766972,153.118957,157.186016,0.524703
std,18710.953092,118.847644,96.096398,95.431617,95.463791,97.310423,95.381686,94.884661,95.395996,0.499393
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
25%,11809.000000,120.000000,82.000000,80.000000,84.000000,83.000000,80.000000,81.000000,85.000000,0.000000
50%,23617.000000,188.000000,139.000000,136.000000,139.000000,139.000000,136.000000,137.000000,141.000000,1.000000
75%,42509.000000,277.000000,208.000000,205.000000,208.000000,210.000000,205.000000,205.000000,210.000000,1.000000
max,66125.000000,585.000000,913.000000,920.000000,933.000000,920.000000,931.000000,933.000000,936.000000,1.000000


In [11]:
df_full.describe(include = 'O')

,period,game-ver,lobby-mode,lobby,mode,stage,A1-weapon,A1-rank,A2-weapon,A2-rank,A3-weapon,A3-rank,A4-weapon,A4-rank,B1-weapon,B1-rank,B2-weapon,B2-rank,B3-weapon,B3-rank,B4-weapon,B4-rank
count,94465,94465,94465,94465,94465,94465,94465,73749,94465,73749,94465,73749,94399,73698,94465,73749,94465,73749,94463,73748,94368,73667
unique,1051,1,2,1,5,23,139,12,139,12,139,12,139,12,139,12,139,12,139,12,139,12
top,2019-10-12T12:00:00+00:00,5.0.1,gachi,standard,area,ama,prime_becchu,x,wakaba,x,prime_becchu,x,splatroller,x,wakaba,x,prime_becchu,x,prime_becchu,x,splatroller,x
freq,351,94465,73749,94465,21123,5311,9416,35319,4064,35319,3870,35319,4716,35291,4215,35319,4194,35319,4464,35319,4802,35278


In [12]:
# 不要な列の削除
train.drop(['game-ver','lobby'], axis=1, inplace=True)
test.drop(['game-ver','lobby'], axis=1, inplace=True) 

In [13]:
# coding:utf-8

#Pandasをインポート
import pandas as pd
import json
from pandas.io.json import json_normalize

#変換したいJSONファイルを読み込む
df = pd.read_json('stage_info.json')
print(df)

# read_jsonした結果だとネストしたjsonを展開できないのでnormalizeで展開させる
df_json = json_normalize(df['data'])
df_json.to_csv("stage_info.csv", encoding='utf-8')
stage_info = pd.read_csv('stage_info.csv')

                                                 data
0   {'key': 'battera', 'splatnet': 0, 'name': {'de...
1   {'key': 'fujitsubo', 'splatnet': 1, 'name': {'...
2   {'key': 'gangaze', 'splatnet': 2, 'name': {'de...
3   {'key': 'kombu', 'splatnet': 5, 'name': {'de_D...
4   {'key': 'ama', 'splatnet': 4, 'name': {'de_DE'...
5   {'key': 'chozame', 'splatnet': 3, 'name': {'de...
6   {'key': 'hokke', 'splatnet': 7, 'name': {'de_D...
7   {'key': 'tachiuo', 'splatnet': 8, 'name': {'de...
8   {'key': 'mystery', 'splatnet': 9999, 'name': {...
9   {'key': 'manta', 'splatnet': 6, 'name': {'de_D...
10  {'key': 'mozuku', 'splatnet': 10, 'name': {'de...
11  {'key': 'engawa', 'splatnet': 9, 'name': {'de_...
12  {'key': 'bbass', 'splatnet': 11, 'name': {'de_...
13  {'key': 'zatou', 'splatnet': 13, 'name': {'de_...
14  {'key': 'arowana', 'splatnet': 15, 'name': {'d...
15  {'key': 'hakofugu', 'splatnet': 14, 'name': {'...
16  {'key': 'devon', 'splatnet': 12, 'name': {'de_...
17  {'key': 'ajifry', 'splat

In [14]:
stage_info.head()

,Unnamed: 0,area,key,name.de_DE,name.en_GB,name.en_US,name.es_ES,name.es_MX,name.fr_CA,name.fr_FR,name.it_IT,name.ja_JP,name.nl_NL,name.ru_RU,name.zh_CN,name.zh_TW,release_at.iso8601,release_at.time,short_name.de_DE,short_name.en_GB,short_name.en_US,short_name.es_ES,short_name.es_MX,short_name.fr_CA,short_name.fr_FR,short_name.it_IT,short_name.ja_JP,short_name.nl_NL,short_name.ru_RU,short_name.zh_CN,short_name.zh_TW,splatnet
0,0,2450.0,battera,Korallenviertel,The Reef,The Reef,Barrio Congrio,Barrio Congrio,Allées salées,Allées salées,Rione Storione,バッテラストリート,Sushistraat,Риф,寿司街,壽司街,2017-03-24T19:00:00+00:00,1490382000,Korallenviertel,Reef,Reef,Barrio Congrio,Barrio Congrio,Allées salées,Allées salées,Rione Storione,バッテラ,Sushistraat,Риф,寿司街,壽司街,0
1,1,1957.0,fujitsubo,Molluskelbude,Musselforge Fitness,Musselforge Fitness,Gimnasio Mejillón,Gimnasio Mejillón,Gymnase Ancrage,Gymnase Ancrage,Centro polpisportivo,フジツボスポーツクラブ,Vinvis Fitness,Спортзал «Кревед!»,藤壶健身房,藤壺健身房,2017-03-24T19:00:00+00:00,1490382000,Molluskelbude,Fitness,Fitness,Gimnasio Mejillón,Gimnasio Mejillón,Gymnase Ancrage,Gymnase Ancrage,Centro polpisportivo,フジツボ,Vinvis Fitness,Спортзал «Кревед!»,健身房,健身房,1
2,2,2140.0,gangaze,Seeigel-Rockbühne,Starfish Mainstage,Starfish Mainstage,Auditorio Erizo,Auditorio Erizo,Scène Sirène,Scène Sirène,Palco Plancton,ガンガゼ野外音楽堂,Zeesterrenstage,КЗ «Иглокожий»,海胆开放音乐台,海膽開放音樂臺,2017-07-15T08:00:00+00:00,1500105600,Seeigel-Rockbühne,Mainstage,Mainstage,Auditorio Erizo,Auditorio Erizo,Scène Sirène,Scène Sirène,Palco Plancton,ガンガゼ,Zeesterrenstage,КЗ «Иглокожий»,音乐台,音樂臺,2
3,3,2259.0,kombu,Buckelwal-Piste,Humpback Pump Track,Humpback Pump Track,Tiburódromo,Tiburódromo,Piste Méroule,Piste Méroule,Tintodromo Montecarpa,コンブトラック,Lekkerbektrack,Велозал «9-й вал»,昆布赛道,昆布賽道,2017-07-15T08:00:00+00:00,1500105600,Buckelwal-Piste,Track,Track,Tiburódromo,Tiburódromo,Piste Méroule,Piste Méroule,Tintodromo Montecarpa,コンブ,Lekkerbektrack,Велозал «9-й вал»,赛道,賽道,5
4,4,2465.0,ama,Perlmutt-Akademie,Inkblot Art Academy,Inkblot Art Academy,Instituto Coralino,Instituto Coralino,Institut Calm'arts,Institut Calam'arts,Campus Hippocampus,海女美術大学,Koraalcampus,Академия «Лепота»,海女美术大学,海女美術大學,2017-07-15T08:00:00+00:00,1500105600,Perlmutt-Akademie,Academy,Academy,Instituto Coralino,Instituto Coralino,Institut Calm'arts,Institut Calam'arts,Campus Hippocampus,アマビ,Koraalcampus,Академия «Лепота»,大学,大學,4


In [15]:
weapon_info = pd.read_csv('statink-weapon2.csv')

In [16]:
weapon_info.head()

,category1,category2,key,subweapon,special,mainweapon,reskin,splatnet,[de-DE],[en-GB],[en-US],[es-ES],[es-MX],[fr-CA],[fr-FR],[it-IT],[ja-JP],[nl-NL],[ru-RU],[zh-CN],[zh-TW]
0,shooter,shooter,52gal,pointsensor,sphere,52gal,52gal,50,.52 Gallon,.52 Gal,.52 Gal,Salpicadora 2000,Salpicadora 2000,Calibre 2000,Calibre 2000,Calibro 2000,.52ガロン,.52 Kaliter,Струевик .52,点52口径重枪,點52口徑重槍
1,shooter,shooter,52gal_deco,curlingbomb,presser,52gal,52gal_deco,51,.52 Gallon Deko,.52 Gal Deco,.52 Gal Deco,Salpicadora 2000 DX,Salpicadora 2000 chic,Calibre 2000 chic,Calibre 2000 chic,Calibro 2000 DX,.52ガロンデコ,.52 Kaliter Deco,Струевик .52 «Деко»,点52口径重枪 Deco,點52口徑重槍 Deco
2,shooter,shooter,96gal,sprinkler,armor,96gal,96gal,80,.96 Gallon,.96 Gal,.96 Gal,Salpicadora 3000,Salpicadora 3000,Calibre 3000,Calibre 3000,Calibro 3000,.96ガロン,.96 Kaliter,Струевик .96,点96口径重枪,點96口徑重槍
3,shooter,shooter,96gal_deco,splashshield,chakuchi,96gal,96gal_deco,81,.96 Gallon Deko,.96 Gal Deco,.96 Gal Deco,Salpicadora 3000 DX,Salpicadora 3000 chic,Calibre 3000 chic,Calibre 3000 chic,Calibro 3000 DX,.96ガロンデコ,.96 Kaliter Deco,Струевик .96 «Деко»,点96口径重枪 Deco,點96口徑重槍 Deco
4,shooter,shooter,promodeler_mg,kyubanbomb,curlingbomb_pitcher,promodeler_mg,promodeler_mg,30,Airbrush MG,Aerospray MG,Aerospray MG,Aerógrafo pro,Aerógrafo GP,Pouche-pouche,Aérogun,Aerografo,プロモデラーMG,Kladderwerper,Аэроспрей,银色喷枪,銀色噴槍


In [17]:
# 一旦periodとstageを消してみる
train.drop(['period','stage'], axis=1, inplace=True)
test.drop(['period','stage'], axis=1, inplace=True) 

In [18]:
# 武器の辞書を作成
dic = weapon_info.set_index('key')['category2'].to_dict()
dic

{'52gal': 'shooter',
 '52gal_deco': 'shooter',
 '96gal': 'shooter',
 '96gal_deco': 'shooter',
 'promodeler_mg': 'shooter',
 'promodeler_pg': 'shooter',
 'promodeler_rg': 'shooter',
 'jetsweeper_custom': 'shooter',
 'momiji': 'shooter',
 'bottlegeyser_foil': 'shooter',
 'prime_collabo': 'shooter',
 'heroshooter_replica': 'shooter',
 'jetsweeper': 'shooter',
 '52gal_becchu': 'shooter',
 'sshooter_becchu': 'shooter',
 'ochiba': 'shooter',
 'prime_becchu': 'shooter',
 'sharp_neo': 'shooter',
 'bold_neo': 'shooter',
 'nzap83': 'shooter',
 'nzap85': 'shooter',
 'nzap89': 'shooter',
 'octoshooter_replica': 'shooter',
 'sharp': 'shooter',
 'sshooter': 'shooter',
 'wakaba': 'shooter',
 'prime': 'shooter',
 'bold': 'shooter',
 'bold_7': 'shooter',
 'bottlegeyser': 'shooter',
 'sshooter_collabo': 'shooter',
 'hotblaster': 'blaster',
 'clashblaster': 'blaster',
 'clashblaster_neo': 'blaster',
 'hotblaster_custom': 'blaster',
 'longblaster_custom': 'blaster',
 'longblaster_necro': 'blaster',
 'hero

In [19]:
# 武器をカテゴリーに置換
train['A1-weapon'] = train['A1-weapon'].replace(dic)
train['A2-weapon'] = train['A2-weapon'].replace(dic)
train['A3-weapon'] = train['A3-weapon'].replace(dic)
train['A4-weapon'] = train['A4-weapon'].replace(dic)
train['B1-weapon'] = train['B1-weapon'].replace(dic)
train['B2-weapon'] = train['B2-weapon'].replace(dic)
train['B3-weapon'] = train['B3-weapon'].replace(dic)
train['B4-weapon'] = train['B4-weapon'].replace(dic)

In [20]:
train.head()

,id,lobby-mode,mode,A1-weapon,A1-rank,A1-level,A2-weapon,A2-rank,A2-level,A3-weapon,A3-rank,A3-level,A4-weapon,A4-rank,A4-level,B1-weapon,B1-rank,B1-level,B2-weapon,B2-rank,B2-level,B3-weapon,B3-rank,B3-level,B4-weapon,B4-rank,B4-level,y
0,1,regular,nawabari,shooter,NaN,139,charger,NaN,118.0,brush,NaN,13.0,brush,NaN,10.0,shooter,NaN,28,brush,NaN,26.0,charger,NaN,68.0,shooter,NaN,31.0,1
1,2,regular,nawabari,brella,NaN,198,shooter,NaN,77.0,brella,NaN,198.0,shooter,NaN,123.0,shooter,NaN,83,charger,NaN,118.0,brella,NaN,168.0,maneuver,NaN,151.0,0
2,3,gachi,hoko,shooter,a-,114,maneuver,a,68.0,shooter,a,225.0,shooter,a,107.0,shooter,a-,50,shooter,a+,163.0,shooter,a-,160.0,maneuver,a,126.0,0
3,4,regular,nawabari,charger,NaN,336,roller,NaN,131.0,roller,NaN,189.0,shooter,NaN,41.0,charger,NaN,273,charger,NaN,189.0,shooter,NaN,194.0,blaster,NaN,391.0,0
4,5,gachi,hoko,shooter,x,299,slosher,x,97.0,reelgun,x,96.0,roller,x,136.0,slosher,x,101,maneuver,x,45.0,slosher,x,246.0,shooter,x,160.0,1


In [21]:
# One-Hotエンコーディング
train = pd.get_dummies(train, columns=['lobby-mode','A1-rank','A2-rank','A3-rank','A4-rank','B1-rank','B2-rank','B3-rank','B4-rank','A1-weapon','A2-weapon','A3-weapon','A4-weapon','B1-weapon','B2-weapon','B3-weapon','B4-weapon'])
test = pd.get_dummies(test, columns=['lobby-mode','A1-rank','A2-rank','A3-rank','A4-rank','B1-rank','B2-rank','B3-rank','B4-rank','A1-weapon','A2-weapon','A3-weapon','A4-weapon','B1-weapon','B2-weapon','B3-weapon','B4-weapon'])

In [22]:
X_train = train.drop(['y'], axis=1) # X_trainはtrainのSurvived列以外
y_train = train['y'] # y_trainはtrainのSurvived列

In [23]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# X_trainとy_trainをtrainとvalidに分割
train_x, valid_x, train_y, valid_y = train_test_split(X_train, y_train,
test_size=0.33, random_state=0)
# lgb.Datasetでtrainとvalidを作っておく
lgb_train = lgb.Dataset(train_x, train_y)
lgb_eval = lgb.Dataset(valid_x, valid_y)
# パラメータを定義
lgbm_params = {'objective': 'binary'}
# lgb.trainで学習
evals_result = {}
gbm = lgb.train(params=lgbm_params,
 train_set=lgb_train,
 valid_sets=[lgb_train, lgb_eval],
 early_stopping_rounds=20,
 evals_result=evals_result,
 verbose_eval=10);

ModuleNotFoundError: No module named 'lightgbm'

In [25]:
import sys
import pprint
pprint.pprint(sys.path)

['/Users/ueshimatsukasa/splatoon',
 '/Library/Frameworks/Python.framework/Versions/3.7/lib/python37.zip',
 '/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7',
 '/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/lib-dynload',
 '',
 '/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages',
 '/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/extensions',
 '/Users/ueshimatsukasa/.ipython']


In [1]:
import sys
print(sys.path)
import lightgbm

['/Users/ueshimatsukasa/splatoon', '/Library/Frameworks/Python.framework/Versions/3.7/lib/python37.zip', '/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7', '/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/lib-dynload', '', '/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages', '/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/extensions', '/Users/ueshimatsukasa/.ipython']


ModuleNotFoundError: No module named 'lightgbm'